In [1]:
import csv

from sklearn.linear_model import Lasso
import numpy as np
from sklearn.metrics import mean_squared_error,mean_absolute_error

import warnings
warnings.filterwarnings('ignore')

TEST_DATA_FILE_1 = "test_data_1.csv"
TEST_DATA_FILE_2 = "test_data_2.csv"
TRAIN_DATA_FILE_1 = "train_data_1.csv"
TRAIN_DATA_FILE_2 = "train_data_2.csv"
PROD_INDEX = 6
DIR_INDEX = 5
ERROR_THRESHOLD_1 = 0
ERROR_THRESHOLD_2 = 0.2
ERROR_THRESHOLD_3 = 0.4

In [2]:
def read_data_file(filename):    
    data_file = file(filename, 'r')
    data = csv.reader(data_file, delimiter=',')
    labels = []
    values = []
    for row in data:
        value = []
        prod_value = 0
        writer_value = 0
        for index in range(1,PROD_INDEX):
            value.append(float(row[index]))
        for index in range(PROD_INDEX + 1,PROD_INDEX + int(row[PROD_INDEX]) + 1):
            prod_value = max(float(row[index]),prod_value)
        value.append(prod_value)
        WRITER_INDEX = PROD_INDEX + int(row[PROD_INDEX]) + 1
        for index in range(WRITER_INDEX+1,len(row)):
            writer_value = max(float(row[index]),writer_value)
        value.append(writer_value)
        labels.append(float(row[0]))
        values.append(value)
    print 'Done Reading File',filename
    return labels,values

In [3]:
train_data_1_labels, train_data_1_values = read_data_file(TRAIN_DATA_FILE_1)
train_data_2_labels, train_data_2_values = read_data_file(TRAIN_DATA_FILE_2)

test_data_1_labels, test_data_1_values = read_data_file(TEST_DATA_FILE_1)
test_data_2_labels, test_data_2_values = read_data_file(TEST_DATA_FILE_2)

Done Reading File train_data_1.csv
Done Reading File train_data_2.csv
Done Reading File test_data_1.csv
Done Reading File test_data_2.csv


In [4]:
def get_error_metric(threshold,actual_labels,predicted_labels):
    error_count = 0
    errors = np.zeros(len(actual_labels))
    for index in range(0,len(actual_labels)):
        errors[index] = (abs(float(actual_labels[index])-float(predicted_labels[index])))
        if abs(float(actual_labels[index])-float(predicted_labels[index])) > threshold:
            error_count +=1
    return error_count*100/len(actual_labels),np.mean(errors),np.std(errors)

def get_mean_squared_error(actual_labels,predicted_labels):
    return mean_squared_error(actual_labels,predicted_labels)

def get_mean_absolute_error(actual_labels,predicted_labels):
    return mean_absolute_error(actual_labels,predicted_labels)

In [5]:
def get_predictions(classifier,val,data):
    if val==1:
        X = np.asarray(train_data_1_values)
        Y = np.asarray(train_data_1_labels)
    else:
        X = np.asarray(train_data_2_values)
        Y = np.asarray(train_data_2_labels)
    classifier.fit(X,Y)
    return classifier.predict(data)

In [6]:
classifier = Lasso()
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)
print 'Mean Absolute Error Mechansim-1:',get_mean_absolute_error(test_data_1_labels,obs_labels)

Lambda:1
Errors for Mechanism-1: Zero Tolerance: 100 0.850173415149 0.660573005044
Errors for Mechanism-1: 0.1 Tolerance: 85 0.850173415149 0.660573005044
Errors for Mechanism-1: 0.2 Tolerance: 70 0.850173415149 0.660573005044
MSE Mechansim-1: 1.15915153082
Mean Absolute Error Mechansim-1: 0.850173415149


In [7]:
classifier = Lasso(alpha=-1)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:-1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)
print 'Mean Absolute Error Mechansim-1:',get_mean_absolute_error(test_data_1_labels,obs_labels)

Lambda:-1
Errors for Mechanism-1: Zero Tolerance: 100 3.96599725137 3.55953396652
Errors for Mechanism-1: 0.1 Tolerance: 95 3.96599725137 3.55953396652
Errors for Mechanism-1: 0.2 Tolerance: 91 3.96599725137 3.55953396652
MSE Mechansim-1: 28.3994162567
Mean Absolute Error Mechansim-1: 3.96599725137


In [8]:
classifier = Lasso(alpha=0.1)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:0.1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)
print 'Mean Absolute Error Mechansim-1:',get_mean_absolute_error(test_data_1_labels,obs_labels)

classifier = Lasso(alpha=2)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:2'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)
print 'Mean Absolute Error Mechansim-1:',get_mean_absolute_error(test_data_1_labels,obs_labels)    

Lambda:0.1
Errors for Mechanism-1: Zero Tolerance: 100 0.284717103478 0.294940771994
Errors for Mechanism-1: 0.1 Tolerance: 48 0.284717103478 0.294940771994
Errors for Mechanism-1: 0.2 Tolerance: 25 0.284717103478 0.294940771994
MSE Mechansim-1: 0.168053887997
Mean Absolute Error Mechansim-1: 0.284717103478
Lambda:2
Errors for Mechanism-1: Zero Tolerance: 100 0.850173415149 0.660573005044
Errors for Mechanism-1: 0.1 Tolerance: 85 0.850173415149 0.660573005044
Errors for Mechanism-1: 0.2 Tolerance: 70 0.850173415149 0.660573005044
MSE Mechansim-1: 1.15915153082
Mean Absolute Error Mechansim-1: 0.850173415149


In [9]:
classifier = Lasso(alpha=-0.1)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:-0.1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)
print 'Mean Absolute Error Mechansim-1:',get_mean_absolute_error(test_data_1_labels,obs_labels)


classifier = Lasso(alpha=0.2)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:0.2'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)
print 'Mean Absolute Error Mechansim-1:',get_mean_absolute_error(test_data_1_labels,obs_labels)


classifier = Lasso(alpha=-0.2)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:-0.2'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)
print 'Mean Absolute Error Mechansim-1:',get_mean_absolute_error(test_data_1_labels,obs_labels)

classifier = Lasso(alpha=0.05)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:0.05'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)
print 'Mean Absolute Error Mechansim-1:',get_mean_absolute_error(test_data_1_labels,obs_labels)

classifier = Lasso(alpha=-0.05)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:-0.05'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)
print 'Mean Absolute Error Mechansim-1:',get_mean_absolute_error(test_data_1_labels,obs_labels)


Lambda:-0.1
Errors for Mechanism-1: Zero Tolerance: 100 0.529409702179 0.481409912521
Errors for Mechanism-1: 0.1 Tolerance: 72 0.529409702179 0.481409912521
Errors for Mechanism-1: 0.2 Tolerance: 50 0.529409702179 0.481409912521
MSE Mechansim-1: 0.512030136635
Mean Absolute Error Mechansim-1: 0.529409702179
Lambda:0.2
Errors for Mechanism-1: Zero Tolerance: 100 0.331894595629 0.313931364457
Errors for Mechanism-1: 0.1 Tolerance: 56 0.331894595629 0.313931364457
Errors for Mechanism-1: 0.2 Tolerance: 30 0.331894595629 0.313931364457
MSE Mechansim-1: 0.208706924198
Mean Absolute Error Mechansim-1: 0.331894595629
Lambda:-0.2
Errors for Mechanism-1: Zero Tolerance: 100 0.933319905476 0.833047829679
Errors for Mechanism-1: 0.1 Tolerance: 83 0.933319905476 0.833047829679
Errors for Mechanism-1: 0.2 Tolerance: 69 0.933319905476 0.833047829679
MSE Mechansim-1: 1.56505473249
Mean Absolute Error Mechansim-1: 0.933319905476
Lambda:0.05
Errors for Mechanism-1: Zero Tolerance: 100 0.278601447351 0

In [10]:
classifier = Lasso()
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)
print 'Mean Absolute Error Mechansim-2:',get_mean_absolute_error(test_data_2_labels,obs_labels)

Lambda:1
Errors for Mechanism-2: Zero Tolerance: 100 0.867761932636 0.71859028245
Errors for Mechanism-2: 0.1 Tolerance: 84 0.867761932636 0.71859028245
Errors for Mechanism-2: 0.2 Tolerance: 69 0.867761932636 0.71859028245
MSE Mechansim-2: 1.26938276576
Mean Absolute Error Mechansim-2: 0.867761932636


In [11]:
classifier = Lasso(alpha=-1)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:-1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)
print 'Mean Absolute Error Mechansim-2:',get_mean_absolute_error(test_data_2_labels,obs_labels)    

Lambda:-1
Errors for Mechanism-2: Zero Tolerance: 100 5.7017191692 5.18899827419
Errors for Mechanism-2: 0.1 Tolerance: 96 5.7017191692 5.18899827419
Errors for Mechanism-2: 0.2 Tolerance: 93 5.7017191692 5.18899827419
MSE Mechansim-2: 59.4353045739
Mean Absolute Error Mechansim-2: 5.7017191692


In [10]:
classifier = Lasso(alpha=0.1)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:0.1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)
print 'Mean Absolute Error Mechansim-2:',get_mean_absolute_error(test_data_2_labels,obs_labels)  

classifier = Lasso(alpha=0.2)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:0.2'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)
print 'Mean Absolute Error Mechansim-2:',get_mean_absolute_error(test_data_2_labels,obs_labels) 

classifier = Lasso(alpha=0.05)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:0.05'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)
print 'Mean Absolute Error Mechansim-2:',get_mean_absolute_error(test_data_2_labels,obs_labels)  


classifier = Lasso(alpha=-0.05)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:-0.05'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)
print 'Mean Absolute Error Mechansim-2:',get_mean_absolute_error(test_data_2_labels,obs_labels)  


classifier = Lasso(alpha=-0.2)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:-0.2'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)
print 'Mean Absolute Error Mechansim-2:',get_mean_absolute_error(test_data_2_labels,obs_labels) 

classifier = Lasso(alpha=-0.1)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:-0.1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)
print 'Mean Absolute Error Mechansim-2:',get_mean_absolute_error(test_data_2_labels,obs_labels) 

Lambda:0.1
Errors for Mechanism-2: Zero Tolerance: 100 0.316874468988 0.328446923051
Errors for Mechanism-2: 0.1 Tolerance: 51 0.316874468988 0.328446923051
Errors for Mechanism-2: 0.2 Tolerance: 29 0.316874468988 0.328446923051
MSE Mechansim-2: 0.208286810358
Mean Absolute Error Mechansim-2: 0.316874468988
Lambda:0.2
Errors for Mechanism-2: Zero Tolerance: 100 0.370162028196 0.353439552185
Errors for Mechanism-2: 0.1 Tolerance: 58 0.370162028196 0.353439552185
Errors for Mechanism-2: 0.2 Tolerance: 35 0.370162028196 0.353439552185
MSE Mechansim-2: 0.261939444167
Mean Absolute Error Mechansim-2: 0.370162028196
Lambda:0.05
Errors for Mechanism-2: Zero Tolerance: 100 0.300154792065 0.298871933188
Errors for Mechanism-2: 0.1 Tolerance: 50 0.300154792065 0.298871933188
Errors for Mechanism-2: 0.2 Tolerance: 26 0.300154792065 0.298871933188
MSE Mechansim-2: 0.179417331647
Mean Absolute Error Mechansim-2: 0.300154792065
Lambda:-0.05
Errors for Mechanism-2: Zero Tolerance: 100 0.404499144617 

In [11]:
avg_rating = 6.031

def read_data_file_2(filename):    
    data_file = file(filename, 'r')
    data = csv.reader(data_file, delimiter=',')
    labels = []
    values = []
    for row in data:
        value = []
        prod_value = 0
        writer_value = 0
        for index in range(1,PROD_INDEX):
            value.append(float(row[index]))
        for index in range(PROD_INDEX + 1,PROD_INDEX + int(row[PROD_INDEX]) + 1):
            prod_value = max(float(row[index]),prod_value)
        value.append(prod_value)
        WRITER_INDEX = PROD_INDEX + int(row[PROD_INDEX]) + 1
        for index in range(WRITER_INDEX+1,len(row)):
            writer_value = max(float(row[index]),writer_value)
        value.append(writer_value)
        labels.append(float(row[0])-avg_rating)
        values.append(value)
    print 'Done Reading File',filename
    return labels,values

train_data_1_labels, train_data_1_values = read_data_file_2(TRAIN_DATA_FILE_1)
train_data_2_labels, train_data_2_values = read_data_file_2(TRAIN_DATA_FILE_2)

test_data_1_labels, test_data_1_values = read_data_file_2(TEST_DATA_FILE_1)
test_data_2_labels, test_data_2_values = read_data_file_2(TEST_DATA_FILE_2)

Done Reading File train_data_1.csv
Done Reading File train_data_2.csv
Done Reading File test_data_1.csv
Done Reading File test_data_2.csv


In [15]:
classifier = Lasso()
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors(2) for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)
print '(2) Mean Absolute Error Mechansim-1:',get_mean_absolute_error(test_data_1_labels,obs_labels) 

classifier = Lasso(alpha=-1)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:-1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors(2) for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)
print '(2) Mean Absolute Error Mechansim-1:',get_mean_absolute_error(test_data_1_labels,obs_labels) 

Lambda:1
Errors(2) for Mechanism-1: Zero Tolerance: 100 0.850173415149 0.660573005044
Errors(2) for Mechanism-1: 0.1 Tolerance: 85 0.850173415149 0.660573005044
Errors(2) for Mechanism-1: 0.2 Tolerance: 70 0.850173415149 0.660573005044
(2) MSE Mechansim-1: 1.15915153082
(2) Mean Absolute Error Mechansim-1: 0.850173415149
Lambda:-1
Errors(2) for Mechanism-1: Zero Tolerance: 100 3.96599725137 3.55953396652
Errors(2) for Mechanism-1: 0.1 Tolerance: 95 3.96599725137 3.55953396652
Errors(2) for Mechanism-1: 0.2 Tolerance: 91 3.96599725137 3.55953396652
(2) MSE Mechansim-1: 28.3994162567
(2) Mean Absolute Error Mechansim-1: 3.96599725137


In [17]:
classifier = Lasso(alpha=-0.1)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:-0.1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors(2) for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)
print '(2) Mean Absolute Error Mechansim-1:',get_mean_absolute_error(test_data_1_labels,obs_labels) 

classifier = Lasso(alpha=0.1)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:0.1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors(2) for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)
print '(2) Mean Absolute Error Mechansim-1:',get_mean_absolute_error(test_data_1_labels,obs_labels)

Lambda:-0.1
Errors(2) for Mechanism-1: Zero Tolerance: 100 0.529409702179 0.481409912521
Errors(2) for Mechanism-1: 0.1 Tolerance: 72 0.529409702179 0.481409912521
Errors(2) for Mechanism-1: 0.2 Tolerance: 50 0.529409702179 0.481409912521
(2) MSE Mechansim-1: 0.512030136635
(2) Mean Absolute Error Mechansim-1: 0.529409702179
Lambda:0.1
Errors(2) for Mechanism-1: Zero Tolerance: 100 0.284717103478 0.294940771994
Errors(2) for Mechanism-1: 0.1 Tolerance: 48 0.284717103478 0.294940771994
Errors(2) for Mechanism-1: 0.2 Tolerance: 25 0.284717103478 0.294940771994
(2) MSE Mechansim-1: 0.168053887997
(2) Mean Absolute Error Mechansim-1: 0.284717103478


In [18]:
classifier = Lasso(alpha=-0.01)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:-0.01'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors(2) for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)
print '(2) Mean Absolute Error Mechansim-1:',get_mean_absolute_error(test_data_1_labels,obs_labels) 

classifier = Lasso(alpha=0.01)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:0.01'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors(2) for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)
print '(2) Mean Absolute Error Mechansim-1:',get_mean_absolute_error(test_data_1_labels,obs_labels)

Lambda:-0.01
Errors(2) for Mechanism-1: Zero Tolerance: 100 0.28347572494 0.272398138074
Errors(2) for Mechanism-1: 0.1 Tolerance: 50 0.28347572494 0.272398138074
Errors(2) for Mechanism-1: 0.2 Tolerance: 24 0.28347572494 0.272398138074
(2) MSE Mechansim-1: 0.154559232256
(2) Mean Absolute Error Mechansim-1: 0.28347572494
Lambda:0.01
Errors(2) for Mechanism-1: Zero Tolerance: 100 0.279901005736 0.27338792212
Errors(2) for Mechanism-1: 0.1 Tolerance: 49 0.279901005736 0.27338792212
Errors(2) for Mechanism-1: 0.2 Tolerance: 24 0.279901005736 0.27338792212
(2) MSE Mechansim-1: 0.153085528973
(2) Mean Absolute Error Mechansim-1: 0.279901005736


In [19]:
classifier = Lasso(alpha=-0.02)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:-0.02'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors(2) for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)
print '(2) Mean Absolute Error Mechansim-1:',get_mean_absolute_error(test_data_1_labels,obs_labels) 

classifier = Lasso(alpha=-2)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:-2'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors(2) for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)
print '(2) Mean Absolute Error Mechansim-2:',get_mean_absolute_error(test_data_1_labels,obs_labels)

Lambda:-0.02
Errors(2) for Mechanism-1: Zero Tolerance: 100 0.290339434498 0.273906491987
Errors(2) for Mechanism-1: 0.1 Tolerance: 52 0.290339434498 0.273906491987
Errors(2) for Mechanism-1: 0.2 Tolerance: 25 0.290339434498 0.273906491987
(2) MSE Mechansim-1: 0.159321753577
(2) Mean Absolute Error Mechansim-1: 0.290339434498
Lambda:-2
Errors(2) for Mechanism-1: Zero Tolerance: 100 7.91423599605 7.11859666035
Errors(2) for Mechanism-1: 0.1 Tolerance: 97 7.91423599605 7.11859666035
Errors(2) for Mechanism-1: 0.2 Tolerance: 95 7.91423599605 7.11859666035
(2) MSE Mechansim-1: 113.309549814
(2) Mean Absolute Error Mechansim-1: 7.91423599605


In [20]:
classifier = Lasso()
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)
print '(2) Mean Absolute Error Mechansim-2:',get_mean_absolute_error(test_data_2_labels,obs_labels)


classifier = Lasso(alpha=-1)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:-1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)
print '(2) Mean Absolute Error Mechansim-2:',get_mean_absolute_error(test_data_2_labels,obs_labels)

Lambda:1
Errors(2) for Mechanism-2: Zero Tolerance: 100 0.867761932636 0.71859028245
Errors(2) for Mechanism-2: 0.1 Tolerance: 84 0.867761932636 0.71859028245
Errors(2) for Mechanism-2: 0.2 Tolerance: 69 0.867761932636 0.71859028245
(2) MSE Mechansim-2: 1.26938276576
(2) Mean Absolute Error Mechansim-1: 0.867761932636
Lambda:-1
Errors(2) for Mechanism-2: Zero Tolerance: 100 5.7017191692 5.18899827419
Errors(2) for Mechanism-2: 0.1 Tolerance: 96 5.7017191692 5.18899827419
Errors(2) for Mechanism-2: 0.2 Tolerance: 93 5.7017191692 5.18899827419
(2) MSE Mechansim-2: 59.4353045739
(2) Mean Absolute Error Mechansim-1: 5.7017191692


In [21]:
classifier = Lasso(alpha=0.1)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:0.1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)
print '(2) Mean Absolute Error Mechansim-2:',get_mean_absolute_error(test_data_2_labels,obs_labels)

classifier = Lasso(alpha=-0.1)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:-0.1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)
print '(2) Mean Absolute Error Mechansim-2:',get_mean_absolute_error(test_data_2_labels,obs_labels)

Lambda:0.1
Errors(2) for Mechanism-2: Zero Tolerance: 100 0.316874468988 0.328446923051
Errors(2) for Mechanism-2: 0.1 Tolerance: 51 0.316874468988 0.328446923051
Errors(2) for Mechanism-2: 0.2 Tolerance: 29 0.316874468988 0.328446923051
(2) MSE Mechansim-2: 0.208286810358
(2) Mean Absolute Error Mechansim-2: 0.316874468988
Lambda:-0.1
Errors(2) for Mechanism-2: Zero Tolerance: 100 0.631459956173 0.560248713095
Errors(2) for Mechanism-2: 0.1 Tolerance: 77 0.631459956173 0.560248713095
Errors(2) for Mechanism-2: 0.2 Tolerance: 57 0.631459956173 0.560248713095
(2) MSE Mechansim-2: 0.712620296774
(2) Mean Absolute Error Mechansim-2: 0.631459956173


In [22]:
classifier = Lasso(alpha=0.01)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:0.01'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)
print '(2) Mean Absolute Error Mechansim-2:',get_mean_absolute_error(test_data_2_labels,obs_labels)

classifier = Lasso(alpha=-0.01)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:-0.01'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)
print '(2) Mean Absolute Error Mechansim-2:',get_mean_absolute_error(test_data_2_labels,obs_labels)

Lambda:0.01
Errors(2) for Mechanism-2: Zero Tolerance: 100 0.299437555529 0.281101344539
Errors(2) for Mechanism-2: 0.1 Tolerance: 54 0.299437555529 0.281101344539
Errors(2) for Mechanism-2: 0.2 Tolerance: 25 0.299437555529 0.281101344539
(2) MSE Mechansim-2: 0.168680815562
(2) Mean Absolute Error Mechansim-2: 0.299437555529
Lambda:-0.01
Errors(2) for Mechanism-2: Zero Tolerance: 100 0.304193764981 0.277304547179
Errors(2) for Mechanism-2: 0.1 Tolerance: 56 0.304193764981 0.277304547179
Errors(2) for Mechanism-2: 0.2 Tolerance: 25 0.304193764981 0.277304547179
(2) MSE Mechansim-2: 0.16943165854
(2) Mean Absolute Error Mechansim-2: 0.304193764981


In [24]:
classifier = Lasso(alpha=-2)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:-2'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)
print '(2) Mean Absolute Error Mechansim-2:',get_mean_absolute_error(test_data_2_labels,obs_labels)

Lambda:-2
Errors(2) for Mechanism-2: Zero Tolerance: 100 11.4024179163 10.3615843692
Errors(2) for Mechanism-2: 0.1 Tolerance: 98 11.4024179163 10.3615843692
Errors(2) for Mechanism-2: 0.2 Tolerance: 96 11.4024179163 10.3615843692
(2) MSE Mechansim-2: 237.377564979
(2) Mean Absolute Error Mechansim-2: 11.4024179163
